In [ ]:
from distutils.command.install import install
! pip install matplotlib

In [ ]:
! brew install pandoc  

In [3]:
################################################
# Jegadeesh & Titman (1993) Momentum Portfolio #
# December 2024                                #  
# Lexuan Chen                                  #
################################################

import pandas as pd
import numpy as np
import wrds
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from scipy import stats

In [ ]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

In [ ]:
%%time
###################
# CRSP Block      #
###################
# sql similar to crspmerge macro
# added exchcd=-2,-1,0 to address the issue that stocks temp stopped trading
# without exchcd=-2,-1, 0 the non-trading months will be tossed out in the output
# leading to wrong cumret calculation in momentum step
# Code	Definition
# -2	Halted by the NYSE or AMEX
# -1	Suspended by the NYSE, AMEX, or NASDAQ
# 0	Not Trading on NYSE, AMEX, or NASDAQ
# 1	New York Stock Exchange
# 2	American Stock Exchange

crsp_m = conn.raw_sql("""
                      select a.permno, a.date,
                      b.shrcd, b.exchcd, 
                      a.ret, a.vol, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/2000' and '12/31/2023'
                      and b.exchcd between -2 and 2
                      and b.shrcd between 10 and 11
                      """) 

# Change variable format to int
crsp_m[['permno','shrcd','exchcd']]=\
    crsp_m[['permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['date']=pd.to_datetime(crsp_m['date'])


## Returns of Relative Strength Portfolios
In this part, we replicate Table 2 of Jegadeesh and Titman (1993). 
The relative strength portfolios are formed based on J-month lagged returns and held for K months.
Since this is an univariate portfolio test, we perform decile sorts and portfolio returns are equal-weighted.
Sample period is from Januar2000 to December 2023.

In [ ]:
J_list = [3,6,9,12]
K_list = [3,6,9,12]

In [ ]:
_tmp_crsp = crsp_m[['permno','date','ret']].sort_values(['permno','date'])\
    .set_index('date')
_tmp_crsp['ret']=_tmp_crsp['ret'].fillna(0)
_tmp_crsp['logret']=np.log(1+_tmp_crsp['ret'])

In [ ]:
_tmp_crsp

### Panel A: No skip between portfolio formation and holding period

In [ ]:
umd_series = {}
tmp_umd_series = {}
for J in J_list:
    for K in K_list: 
        umd = _tmp_crsp.groupby('permno')['logret'].rolling(J, min_periods=J).sum().reset_index()  
        umd['cumret']=np.exp(umd['logret'])-1
    
        ########################################
        # Formation of 10 Momentum Portfolios  #
        ########################################
        umd=umd.dropna(axis=0, subset=['cumret'])
        umd['momr']=umd.groupby('date')['cumret'].transform(lambda x: pd.qcut(x, 10, labels=False))
        umd['momr'] = umd['momr'].astype(int) 
        umd['momr'] = umd['momr']+1
    
        umd['form_date'] = umd['date']
        umd['medate'] = umd['date']+MonthEnd(0)
        umd['hdate1']=umd['medate']+MonthBegin(1) 
        umd['hdate2']=umd['medate']+MonthEnd(K)   ###是这里不一样！！！

        tmp1 = umd
        tmp1['hdate1_m'] = pd.DatetimeIndex(tmp1['hdate1']).month
        tmp1['hdate2_m'] = pd.DatetimeIndex(tmp1['hdate2']).month
        
        #Calculate the monthly difference (mgap) of the holding period and deal with cross-year situations
        tmp1['mgap'] = tmp1.hdate2_m - tmp1.hdate1_m
        tmp1['mgap'] = np.where(tmp1.mgap>0, tmp1.mgap, tmp1.mgap+12)
        tmp_umd_series[(J,K)] = tmp1

        umd = umd[['permno','form_date','momr','hdate1','hdate2']]
        umd_series[(J,K)] = umd

In [ ]:
Panel_A = {}
_tmp_ret = crsp_m[['permno','date','ret']]
_tmp_ret['ret'] = _tmp_ret['ret'].fillna(0) # 像这一行 其实前面最开始处理的时候你已经处理过了 你又搞一遍

for J in J_list:
    for K in K_list: 
        umd = umd_series[(K,J)]
        
        port = pd.DataFrame()
        n = 100000
        
        for i in range(0, _tmp_ret.shape[0], n): 
            
            chunks =   _tmp_ret.iloc[i:i + n]
            merged = umd.merge(chunks, on=['permno'], how='inner')
            merged = merged[(merged['hdate1']<=merged['date']) & (merged['date']<=merged['hdate2'])]
            port = pd.concat([port, merged],ignore_index=True )
        
        umd2 = port.sort_values(by=['date','momr','form_date','permno']).drop_duplicates()
        
        umd3 = umd2.groupby(['date','momr','form_date'])['ret'].mean().reset_index()
        start_yr = umd3['date'].dt.year.min()+2
        umd3 = umd3[umd3['date'].dt.year>=start_yr]
        umd3 = umd3.sort_values(by=['date','momr'])
        
        # Create one return series per MOM group every month
        ewret = umd3.groupby(['date','momr'])['ret'].mean().reset_index()
        ewstd = umd3.groupby(['date','momr'])['ret'].std().reset_index()
        ewret = ewret.rename(columns={'ret':'ewret'})
        ewstd = ewstd.rename(columns={'ret':'ewretstd'})
        ewretdat = pd.merge(ewret, ewstd, on=['date','momr'], how='inner')
        ewretdat = ewretdat.sort_values(by=['momr'])
        
        # Transpose portfolio layout to have columns as portfolio returns
        ewretdat2 = ewretdat.pivot(index='date', columns='momr', values='ewret')
        
        # Add prefix port in front of each column
        ewretdat2 = ewretdat2.add_prefix('port') 
        ewretdat2 = ewretdat2.rename(columns={'port1':'losers', 'port10':'winners'})
        ewretdat2['long_short'] = ewretdat2['winners'] - ewretdat2['losers']
        
        # Compute Long-Short Portfolio Cumulative Returns
        ewretdat3 = ewretdat2
        ewretdat3['1+losers']=1+ewretdat3['losers']
        ewretdat3['1+winners']=1+ewretdat3['winners']
        ewretdat3['1+ls'] = 1+ewretdat3['long_short']
        
        #cumprod(): Calculate the cumulative product for each sequence,
        ewretdat3['cumret_winners']=ewretdat3['1+winners'].cumprod()-1
        ewretdat3['cumret_losers']=ewretdat3['1+losers'].cumprod()-1
        ewretdat3['cumret_long_short']=ewretdat3['1+ls'].cumprod()-1
        
        #################################
        # Portfolio Summary Statistics  #
        ################################# 
        
        # Mean 
        mom_mean = ewretdat3[['winners', 'losers', 'long_short']].mean().to_frame()
        mom_mean = mom_mean.rename(columns={0:f'{K}_mean'}).reset_index()
        
        # T-Value and P-Value
        t_losers = pd.Series(stats.ttest_1samp(ewretdat3['losers'],0.0)).to_frame().T
        t_winners = pd.Series(stats.ttest_1samp(ewretdat3['winners'],0.0)).to_frame().T
        t_long_short = pd.Series(stats.ttest_1samp(ewretdat3['long_short'],0.0)).to_frame().T
        
        t_losers['momr']='losers'
        t_winners['momr']='winners'
        t_long_short['momr']='long_short'
        
        t_output =pd.concat([t_winners, t_losers, t_long_short])\
            .rename(columns={0:f'{K}_t-stat', 1:f'{K}_p-value'})
        
        # Combine mean, t and p
        mom_output = pd.merge(mom_mean, t_output, on=['momr'], how='inner')
        mom_output['momr'] = [f'{J} Sell', f'{J} Buy', f'{J} Buy-sell']
        
        mom_output = mom_output[['momr',f'{K}_mean',f'{K}_t-stat']]
        Panel_A[(J,K)] = mom_output

In [ ]:
results1 = pd.concat([Panel_A[(3,3)],Panel_A[(6,3)],Panel_A[(9,3)],Panel_A[(12,3)]],ignore_index=True)
results2 = pd.concat([Panel_A[(3,6)],Panel_A[(6,6)],Panel_A[(9,6)],Panel_A[(12,6)]],ignore_index=True)     
results3 = pd.concat([Panel_A[(3,9)],Panel_A[(6,9)],Panel_A[(9,9)],Panel_A[(12,9)]],ignore_index=True) 
results4 = pd.concat([Panel_A[(3,12)],Panel_A[(6,12)],Panel_A[(9,12)],Panel_A[(12,12)]],ignore_index=True) 

result = pd.merge(results1, results2, on=['momr'], how='inner')
result = pd.merge(result, results3, on=['momr'], how='inner')
result = pd.merge(result, results4, on=['momr'], how='inner')
result.rename(columns={'momr':'J'},inplace=True)
result = result[['J','3_mean','3_t-stat','6_mean','6_t-stat','9_mean','9_t-stat','12_mean','12_t-stat']].round(6)

In [ ]:
result.to_csv('Table 2 Panel_A.csv',index=True)

### Panel B: One-month skip between portfolio formation and holding period

In [ ]:
skip_umd_series = {}
skip_tmp_umd_series = {}
for J in J_list:
    for K in K_list: 
        '''
        umd = _tmp_crsp.groupby('permno')['logret'].rolling(J, min_periods=J-1).sum().reset_index()  
        这里出错了老铁！！！！！！
        '''
        ###应该是
        umd = _tmp_crsp.groupby('permno')['logret'].rolling(J-1, min_periods=J-1).sum().reset_index()

        umd['cumret']=np.exp(umd['logret'])-1
    
        ########################################
        # Formation of 10 Momentum Portfolios  #
        ########################################
        umd=umd.dropna(axis=0, subset=['cumret'])
        umd['momr']=umd.groupby('date')['cumret'].transform(lambda x: pd.qcut(x, 10, labels=False))
        umd['momr'] = umd['momr'].astype(int) 
        umd['momr'] = umd['momr']+1
    
        umd['form_date'] = umd['date']
        umd['medate'] = umd['date']+MonthEnd(0)
        umd['hdate1']=umd['medate']+MonthBegin(1) 
        umd['hdate2']=umd['medate']+MonthEnd(K)

        tmp1 = umd
        tmp1['hdate1_m'] = pd.DatetimeIndex(tmp1['hdate1']).month
        tmp1['hdate2_m'] = pd.DatetimeIndex(tmp1['hdate2']).month
        
        #Calculate the monthly difference (mgap) of the holding period and deal with cross-year situations
        tmp1['mgap'] = tmp1.hdate2_m - tmp1.hdate1_m
        tmp1['mgap'] = np.where(tmp1.mgap>0, tmp1.mgap, tmp1.mgap+12)
        skip_tmp_umd_series[(J,K)] = tmp1

        umd = umd[['permno','form_date','momr','hdate1','hdate2']]
        skip_umd_series[(J,K)] = umd

In [ ]:
Panel_B = {}
_tmp_ret = crsp_m[['permno','date','ret']]
_tmp_ret['ret'] = _tmp_ret['ret'].fillna(0)

for J in J_list:
    for K in K_list: 
        umd = umd_series[(K,J)]
        
        port = pd.DataFrame()
        n = 100000
        
        for i in range(0, _tmp_ret.shape[0], n): 
            
            chunks =   _tmp_ret.iloc[i:i + n]
            merged = umd.merge(chunks, on=['permno'], how='inner')
            merged = merged[(merged['hdate1']<=merged['date']) & (merged['date']<=merged['hdate2'])]
            port = pd.concat([port, merged],ignore_index=True )
        
        umd2 = port.sort_values(by=['date','momr','form_date','permno']).drop_duplicates()
        
        umd3 = umd2.groupby(['date','momr','form_date'])['ret'].mean().reset_index()
        start_yr = umd3['date'].dt.year.min()+2
        #获取数据集中的最小年份，并加上2，表示从 形成期结束后的第二年 开始计算。
        # 这样做是为了避免在形成期初期的数据不够稳定时使用数据。
        
        umd3 = umd3[umd3['date'].dt.year>=start_yr]
        umd3 = umd3.sort_values(by=['date','momr'])
        
        # Create one return series per MOM group every month
        ewret = umd3.groupby(['date','momr'])['ret'].mean().reset_index()
        ewstd = umd3.groupby(['date','momr'])['ret'].std().reset_index()
        ewret = ewret.rename(columns={'ret':'ewret'})
        ewstd = ewstd.rename(columns={'ret':'ewretstd'})
        ewretdat = pd.merge(ewret, ewstd, on=['date','momr'], how='inner')
        ewretdat = ewretdat.sort_values(by=['momr'])
        
        # Transpose portfolio layout to have columns as portfolio returns
        ewretdat2 = ewretdat.pivot(index='date', columns='momr', values='ewret')
        
        # Add prefix port in front of each column
        ewretdat2 = ewretdat2.add_prefix('port') 
        ewretdat2 = ewretdat2.rename(columns={'port1':'losers', 'port10':'winners'})
        ewretdat2['long_short'] = ewretdat2['winners'] - ewretdat2['losers']
        
        # Compute Long-Short Portfolio Cumulative Returns
        ewretdat3 = ewretdat2
        ewretdat3['1+losers']=1+ewretdat3['losers']
        ewretdat3['1+winners']=1+ewretdat3['winners']
        ewretdat3['1+ls'] = 1+ewretdat3['long_short']
        
        #cumprod(): Calculate the cumulative product for each sequence,
        ewretdat3['cumret_winners']=ewretdat3['1+winners'].cumprod()-1
        ewretdat3['cumret_losers']=ewretdat3['1+losers'].cumprod()-1
        ewretdat3['cumret_long_short']=ewretdat3['1+ls'].cumprod()-1
        
        #################################
        # Portfolio Summary Statistics  #
        ################################# 
        
        # Mean 
        mom_mean = ewretdat3[['winners', 'losers', 'long_short']].mean().to_frame()
        mom_mean = mom_mean.rename(columns={0:f'{K}_mean'}).reset_index()
        
        # T-Value and P-Value
        t_losers = pd.Series(stats.ttest_1samp(ewretdat3['losers'],0.0)).to_frame().T
        t_winners = pd.Series(stats.ttest_1samp(ewretdat3['winners'],0.0)).to_frame().T
        t_long_short = pd.Series(stats.ttest_1samp(ewretdat3['long_short'],0.0)).to_frame().T
        
        t_losers['momr']='losers'
        t_winners['momr']='winners'
        t_long_short['momr']='long_short'
        
        t_output =pd.concat([t_winners, t_losers, t_long_short])\
            .rename(columns={0:f'{K}_t-stat', 1:f'{K}_p-value'})
        
        # Combine mean, t and p
        mom_output = pd.merge(mom_mean, t_output, on=['momr'], how='inner')
        mom_output['momr'] = [f'{J} Sell', f'{J} Buy', f'{J} Buy-sell']
        
        mom_output = mom_output[['momr',f'{K}_mean',f'{K}_t-stat']]
        Panel_B[(J,K)] = mom_output

In [122]:
skip_results1 = pd.concat([Panel_B[(3,3)],Panel_B[(6,3)],Panel_B[(9,3)],Panel_B[(12,3)]],ignore_index=True)
skip_results2 = pd.concat([Panel_B[(3,6)],Panel_B[(6,6)],Panel_B[(9,6)],Panel_B[(12,6)]],ignore_index=True)     
skip_results3 = pd.concat([Panel_B[(3,9)],Panel_B[(6,9)],Panel_B[(9,9)],Panel_B[(12,9)]],ignore_index=True) 
skip_results4 = pd.concat([Panel_B[(3,12)],Panel_B[(6,12)],Panel_B[(9,12)],Panel_B[(12,12)]],ignore_index=True) 
skip_result = pd.merge(skip_results1, skip_results2, on=['momr'], how='inner')
skip_result = pd.merge(skip_result, skip_results3, on=['momr'], how='inner')
skip_result = pd.merge(skip_result, skip_results4, on=['momr'], how='inner')
skip_result.rename(columns={'momr':'J'},inplace=True)
skip_result = result[['J','3_mean','3_t-stat','6_mean','6_t-stat','9_mean','9_t-stat','12_mean','12_t-stat']].round(6)

In [124]:
skip_result.to_csv('Table 2 Panel_B.csv',index=True)

## Performance of portfolios double-sorted on recent and intermediate momentum

In this part, we try to replicate Panel A of Table 4 of Novy-Marx (2012)which reports the excess returns(percent per month) of portfolios double-sorted by recent momentum and intermediate momentumwe. To maintain highly diversified portfolios weemploy quintile sorts (again using NYSE breakpoints) and portfolio returns are value-weighted. We use the risk-free rate as the benchmark.Sample period is from January 1927 to December 2023.

In [364]:
conn = wrds.Connection()  

Enter your WRDS username [benben]: lexuan
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [365]:
tables = conn.raw_sql("""  
    SELECT table_name   
    FROM information_schema.tables   
    WHERE table_schema = 'ff';  
""")  

RF = conn.raw_sql("""  
                SELECT dateff, rf            
                FROM "ff"."factors_monthly" 
                WHERE date BETWEEN '1927-01-01' AND '2023-12-31'  
                """)  

RF['dateff'] = pd.to_datetime(RF['dateff']) 
RF['hdate'] = RF['dateff'] + MonthEnd(0)
RF = RF[['hdate','rf']]
RF['hdate'] = pd.to_datetime(RF['hdate']) 

In [366]:
crsp_m = conn.raw_sql("""
                      select a.permno, a.date,
                      b.shrcd, b.exchcd, 
                      a.ret, a.vol, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1927' and '12/31/2023'
                      and b.exchcd between -2 and 2
                      and b.shrcd between 10 and 11
                      """) 

# Change variable format to int
crsp_m[['permno','shrcd','exchcd']]=\
    crsp_m[['permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['date']=pd.to_datetime(crsp_m['date'])

_tmp_crsp = crsp_m[['permno','date','ret']].sort_values(['permno','date'])\
    .set_index('date')
_tmp_crsp['ret']=_tmp_crsp['ret'].fillna(0)
_tmp_crsp['logret']=np.log(1+_tmp_crsp['ret'])

In [368]:
_tmp_crsp['logret']=np.log(1+_tmp_crsp['ret'])

In [369]:
umd_6 = _tmp_crsp.groupby('permno')['logret'].rolling(6, min_periods=6).sum().reset_index()
umd_12 = _tmp_crsp.groupby('permno')['logret'].rolling(12, min_periods=12).sum().reset_index()
umd_6['cumret_6']=np.exp(umd_6['logret'])-1
umd_12['cumret_12']=np.exp(umd_12['logret'])-1

In [370]:
umd = pd.merge(umd_6,umd_12,on=['permno','date'],how='inner')
umd['mid_ret'] = umd['cumret_12']-umd['cumret_6']
umd['recent_ret'] = umd['cumret_6']
umd=umd.dropna(axis=0, subset=['mid_ret','recent_ret']).reset_index()
umd = umd[['permno','date','mid_ret','recent_ret']]

In [371]:
# For each date: assign ranking 1-5 based on intermmediate and most recent 6-month cumulative return respectively
# 1=lowest 5=highest cumret

umd['mid_momr']=umd.groupby('date')['mid_ret'].transform(lambda x: pd.qcut(x, 5, labels=False))
umd['recent_momr']=umd.groupby('date')['recent_ret'].transform(lambda x: pd.qcut(x, 5, labels=False))

umd.mid_momr=umd.mid_momr.astype(int) 
umd['mid_momr'] = umd['mid_momr']+1
umd.recent_momr=umd.recent_momr.astype(int) 
umd['recent_momr'] = umd['recent_momr']+1
umd['form_date'] = umd['date']+ MonthEnd(0)
umd['hdate']=umd['date']+MonthEnd(1)
umd = umd[['permno','date','mid_momr','recent_momr','form_date','hdate']]

In [372]:
_tmp_ret = crsp_m[['permno','date','ret','shrout','prc']]
_tmp_ret = _tmp_ret[(_tmp_ret['prc'].notna()) & (_tmp_ret['shrout'].notna()) & (_tmp_ret['ret'].notna())]
_tmp_ret['hdate']=  _tmp_ret['date']+ MonthEnd(0)

In [373]:
# join rank and other necessary data together
# Merge on 'hdate' i.e. the holding period

_tmp_ret = crsp_m[['permno','date','ret','shrout','prc']]
_tmp_ret = _tmp_ret[(_tmp_ret['prc'].notna()) & (_tmp_ret['shrout'].notna()) & (_tmp_ret['ret'].notna())]
_tmp_ret['hdate']=  _tmp_ret['date']+MonthEnd(0)

port = pd.DataFrame()
n = 100000

for i in range(0, _tmp_ret.shape[0], n): 
    chunks =   _tmp_ret.iloc[i:i + n]
    merged = umd.merge(chunks, on=['permno','hdate'], how='left')
    port = pd.concat([port, merged])

In [374]:
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

# value-weigthed return
port['size'] = port['prc']*port['shrout'].abs()
port_whgtret = port.groupby(['hdate', 'mid_momr','recent_momr']).apply(wavg, 'ret', 'size', include_groups=False).reset_index(name='vwret')  

In [375]:
ER = port_whgtret.merge(RF, on=['hdate'], how='left')
ER['excuss_ret'] = ER['vwret']-ER['rf']

In [376]:
#Get Long_short portfolios in  Recent_momr given Mid_momr
Mid_Portfolio = ER.pivot(index=['mid_momr','hdate'], columns= 'recent_momr', values= 'excuss_ret')
Mid_Portfolio['5-1'] = Mid_Portfolio[5]-Mid_Portfolio[1] 
Mid_Portfolio.dropna(subset=['5-1'], inplace=True)  
Summary_mid = Mid_Portfolio.groupby(level='mid_momr').describe()['5-1'].loc[:, ['mean']]
t_values = Mid_Portfolio.groupby(level='mid_momr')['5-1'].apply(lambda x: stats.ttest_1samp(x, 0).statistic)      
Summary_mid['t-value'] = t_values  

#Get Long_short portfolios in  Mid_momr given Recent_momr
Recent_Portfolio = ER.pivot(index=['recent_momr','hdate'], columns= 'mid_momr', values= 'excuss_ret')
Recent_Portfolio['5-1'] = Recent_Portfolio[5]-Recent_Portfolio[1]
Recent_Portfolio.dropna(subset=['5-1'], inplace=True)  
Summary_recent = Recent_Portfolio.groupby(level='recent_momr').describe()['5-1'] .loc[:, ['mean']] #mid_momr 做差
t_values = Recent_Portfolio.groupby(level='recent_momr')['5-1'].apply(lambda x: stats.ttest_1samp(x, 0).statistic)   
Summary_recent['t-value'] = t_values  

In [377]:
Mean_Output = ER.pivot_table(index='recent_momr', columns='mid_momr', values='vwret', aggfunc='mean')

In [378]:
Summary_T = ER.groupby(['mid_momr', 'recent_momr'])['vwret'].apply(lambda x: stats.ttest_1samp(x, 0).statistic)  
Summary_t_value_df = Summary_T.reset_index(name='t-statistic')  
T_Output = Summary_t_value_df.pivot(index='recent_momr', columns='mid_momr', values='t-statistic')

In [379]:
Mean_Output['5-1'] = Summary_recent['mean']
result_mean = pd.concat([Mean_Output,Summary_mid['mean'].T.to_frame().T],axis=0)
result_mean.columns = ['IR1','IR2','IR3','IR4','IR5','long-short']
new_index = ['RR1','RR2','RR3','RR4','RR5','long-short']
result_mean.index = new_index 
result_mean = (result_mean*100).round(2)
result_mean.name = "Estimate (percent)"

In [380]:
T_Output['5-1'] = Summary_recent['t-value']
result_t = pd.concat([T_Output, Summary_mid['t-value'].T.to_frame().T], axis=0)
result_t.columns = ['IR1','IR2','IR3','IR4','IR5','long-short']
result_t.index = new_index 
result_t = result_t.round(2)
result_t.name = "Test statistics"

In [381]:
final_result = pd.concat([result_mean, result_t], axis=1) 
final_result.columns = pd.MultiIndex.from_tuples(  
    [(result_mean.name, col) for col in result_mean.columns] +  
    [(result_t.name, col) for col in result_t.columns])    
final_result.index.name = None 
final_result.to_csv('Table 4 Panel_A.csv',index=True)

## Brief Analysis and Discussion
Our replicate result reinforces the previously established conclusions, demonstrating that both intermediate horizon performance and recent past performance are significantly correlated with future returns. However, recent past performance exhibits a lesser predictive capability.

The analysis reveals that while return spreads are generally significant, the magnitude and significance of the differences between intermediate horizon winners and losers are notable. For instance, the long-short estimates indicate a positive return for intermediate horizon winners compared to losers, particularly in IR5, where the estimate is 2.61% for winners and 6.19% for losers, suggesting a significant spread.

A direct comparison between recent winners who were intermediate horizon losers and recent losers who had also been intermediate horizon losers reveals interesting dynamics. Specifically, stocks that have significantly increased in value over the past six months (e.g., RR5 in IR1 with a return of 3.07%) but performed poorly in the prior six months tend to underperform relative to stocks that have decreased in value but performed well earlier (e.g., RR4 in IR4 with a return of 1.83%).

The monthly return differential between the intermediate horizon winners – recent losers portfolio and the recent winners – intermediate horizon losers portfolio is recorded at 0.95% (long-short estimate for RR1), with a test statistic of 0.40. This indicates a weaker predictive power for recent past performance compared to intermediate horizon performance.